# PropertyLoop Financial Data Chatbot

**Objective:** Answer questions strictly from `holdings.csv` and `trades.csv` without hallucinations.

**Fallback Response:** `Sorry can not find the answer`

## 1. Setup

In [ ]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
import warnings
from dotenv import load_dotenv
warnings.filterwarnings('ignore')

# Load environment variables
load_dotenv()

# Install groq if not already installed
try:
    from groq import Groq
except ImportError:
    import subprocess
    subprocess.check_call(['pip', 'install', 'groq', '-q'])
    from groq import Groq

# Initialize Groq client
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
if not GROQ_API_KEY:
    print("❌ Error: GROQ_API_KEY not found in .env file")
else:
    groq_client = Groq(api_key=GROQ_API_KEY)
    print("✓ Setup complete")

## 2. Load Data

In [ ]:
holdings_df = pd.read_csv('holdings.csv')
trades_df = pd.read_csv('trades.csv')
print(f"Holdings: {holdings_df.shape[0]} rows | Trades: {trades_df.shape[0]} rows")
display(holdings_df.head(2))
display(trades_df.head(2))

## 3. Preprocess Data

In [ ]:
# Normalize column names and convert data types
holdings_df.columns = holdings_df.columns.str.lower().str.strip()
trades_df.columns = trades_df.columns.str.lower().str.strip()

# Convert dates
for col in ['asofdate', 'opendate', 'closedate']:
    if col in holdings_df.columns:
        holdings_df[col] = pd.to_datetime(holdings_df[col], errors='coerce')

for col in ['tradedate', 'settledate']:
    if col in trades_df.columns:
        trades_df[col] = pd.to_datetime(trades_df[col], errors='coerce')

# Convert numeric columns
for col in ['qty', 'price', 'mv_base', 'pl_ytd']:
    if col in holdings_df.columns:
        holdings_df[col] = pd.to_numeric(holdings_df[col], errors='coerce')

print("✓ Data preprocessed")

## 4. Intent Detection

In [ ]:
class IntentDetector:
    def __init__(self):
        self.patterns = {
            'count_holdings': ['how many holdings', 'total holdings', 'number of holdings'],
            'count_trades': ['how many trades', 'total trades', 'number of trades'],
            'fund_performance': ['which fund performed better', 'best performing fund', 'fund performance'],
            'yearly_pnl': ['yearly p&l', 'yearly profit', 'p&l comparison', 'annual p&l']
        }
    
    def detect(self, query):
        q = query.lower().strip()
        for intent, patterns in self.patterns.items():
            if any(p in q for p in patterns):
                return intent
        return 'unknown'
    
    def extract_fund(self, query):
        words = query.split()
        for i, word in enumerate(words):
            if word.lower() in ['for', 'fund', 'portfolio'] and i + 1 < len(words):
                return ' '.join(words[i+1:i+3]).strip('?.,!')
        return None

## 5. Business Logic

In [ ]:
def count_holdings(df, fund=None):
    if fund:
        matches = df[df['portfolioname'].str.lower().str.contains(fund.lower(), na=False)]
        return (len(matches), matches['portfolioname'].iloc[0]) if len(matches) > 0 else None
    return len(df), "All Funds"

def count_trades(df, fund=None):
    if fund:
        matches = df[df['portfolioname'].str.lower().str.contains(fund.lower(), na=False)]
        return (len(matches), matches['portfolioname'].iloc[0]) if len(matches) > 0 else None
    return len(df), "All Funds"

def yearly_pnl(df):
    if 'portfolioname' not in df.columns or 'pl_ytd' not in df.columns:
        return None
    pnl = df.groupby('portfolioname')['pl_ytd'].sum().reset_index()
    pnl.columns = ['Fund', 'Yearly_PnL']
    pnl = pnl[pnl['Yearly_PnL'].notna() & (pnl['Yearly_PnL'] != 0)].sort_values('Yearly_PnL', ascending=False)
    return pnl

def best_fund(df):
    pnl = yearly_pnl(df)
    return (pnl.iloc[0]['Fund'], pnl.iloc[0]['Yearly_PnL']) if pnl is not None and len(pnl) > 0 else None

## 6. Chatbot with Groq LLM

In [ ]:
class Chatbot:
    FALLBACK = "Sorry can not find the answer"
    
    def __init__(self, holdings, trades, use_llm=True):
        self.holdings = holdings
        self.trades = trades
        self.detector = IntentDetector()
        self.use_llm = use_llm
    
    def format_with_llm(self, data_result, query):
        """Use Groq LLM to format computed results into natural language"""
        try:
            prompt = f"""You are a financial assistant. Format the following data result into a clear, natural language response.

User Question: {query}

Computed Data Result: {data_result}

Instructions:
- Use ONLY the data provided above
- Do NOT add any external information
- Keep response concise and professional
- Format numbers with proper commas and currency symbols where appropriate

Response:"""
            
            response = groq_client.chat.completions.create(
                model="llama-3.3-70b-versatile",
                messages=[{"role": "user", "content": prompt}],
                temperature=0.1,
                max_tokens=500
            )
            
            return response.choices[0].message.content.strip()
        except Exception as e:
            print(f"LLM Error: {e}")
            return self.format_simple(data_result)
    
    def format_simple(self, result):
        """Simple template-based formatting (fallback)"""
        if isinstance(result, tuple):
            if len(result) == 2:
                return f"{result[1]}: {result[0]:,}"
        elif isinstance(result, pd.DataFrame):
            resp = "Yearly P&L:\n"
            for _, row in result.head(10).iterrows():
                resp += f"  • {row['Fund']}: ${row['Yearly_PnL']:,.2f}\n"
            return resp
        return str(result)
    
    def answer(self, query):
        intent = self.detector.detect(query)
        if intent == 'unknown':
            return self.FALLBACK
        
        fund = self.detector.extract_fund(query)
        result = None
        
        if intent == 'count_holdings':
            result = count_holdings(self.holdings, fund)
        elif intent == 'count_trades':
            result = count_trades(self.trades, fund)
        elif intent == 'fund_performance':
            result = best_fund(self.holdings)
        elif intent == 'yearly_pnl':
            result = yearly_pnl(self.holdings)
        
        if result is None or (isinstance(result, pd.DataFrame) and len(result) == 0):
            return self.FALLBACK
        
        if self.use_llm:
            return self.format_with_llm(result, query)
        else:
            return self.format_simple(result)

chatbot = Chatbot(holdings_df, trades_df, use_llm=True)
print("✓ Chatbot ready (Groq LLM enabled)")

## 7. Test Cases

In [ ]:
# Valid queries
test_queries = [
    "How many holdings does Garfield have?",
    "How many trades does Heather have?",
    "Which fund performed better based on yearly Profit and Loss?",
    "Show me the yearly P&L comparison across all funds",
    "What is the total number of holdings?"
]

print("=" * 80)
print("VALID QUERIES (with Groq LLM formatting)")
print("=" * 80)
for q in test_queries:
    print(f"\nQ: {q}")
    print(f"A: {chatbot.answer(q)}")
    print("-" * 80)

In [ ]:
# Invalid queries (should return fallback)
invalid_queries = [
    "How many holdings does XYZ Fund have?",
    "What is the weather today?",
    "Tell me something interesting",
    "What is the current stock price of Apple?"
]

print("=" * 80)
print("INVALID QUERIES (Should return fallback)")
print("=" * 80)
for q in invalid_queries:
    print(f"\nQ: {q}")
    print(f"A: {chatbot.answer(q)}")
    print("-" * 80)

## 8. Interactive Demo

In [ ]:
# Uncomment to run interactive demo
# while True:
#     query = input("You: ").strip()
#     if query.lower() in ['quit', 'exit', 'q']:
#         break
#     if query:
#         print(f"Bot: {chatbot.answer(query)}\n")